###讀取資料

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import pandas as pd
##新聞、dcard、ptt 段詞結果
dcard_WS=pd.read_excel("/content/drive/MyDrive/大學/雲端-野村投信/斷詞完畢/dcard_WS.xlsx")
headline=pd.read_excel("/content/drive/MyDrive/大學/雲端-野村投信/斷詞完畢/headline.xlsx")
ptt_WS=pd.read_excel("/content/drive/MyDrive/大學/雲端-野村投信/斷詞完畢/ptt_WS.xlsx")
wdstock=pd.read_csv("/content/drive/MyDrive/大學/雲端-野村投信/斷詞完畢/wdstock.csv")

##持股整理後資料
nnl=pd.read_excel("/content/drive/MyDrive/大學/雲端-野村投信/野村持股(修改版)/NN (L)_修改版.xlsx")
abran=pd.read_excel("/content/drive/MyDrive/大學/雲端-野村投信/野村持股(修改版)/安本標準_修改版.xlsx")
ninetyone=pd.read_excel("/content/drive/MyDrive/大學/雲端-野村投信/野村持股(修改版)/晉達_修改版.xlsx")
ubsag=pd.read_excel("/content/drive/MyDrive/大學/雲端-野村投信/野村持股(修改版)/瑞銀_修改版.xlsx")
roboco=pd.read_excel("/content/drive/MyDrive/大學/雲端-野村投信/野村持股(修改版)/荷寶_修改版.xlsx")
ireland=pd.read_excel("/content/drive/MyDrive/大學/雲端-野村投信/野村持股(修改版)/野村基金(愛爾蘭系列)_修改版.xlsx")
nomura=pd.read_excel("/content/drive/MyDrive/大學/雲端-野村投信/野村持股(修改版)/野村投信_修改版.xlsx")
amundi=pd.read_excel("/content/drive/MyDrive/大學/雲端-野村投信/野村持股(修改版)/鋒裕匯理_修改版.xlsx")
janus=pd.read_excel("/content/drive/MyDrive/大學/雲端-野村投信/野村持股(修改版)/駿利亨德森_修改版.xlsx")

##情緒字典
moods=pd.read_excel("/content/drive/MyDrive/大學/雲端-野村投信/情緒字典_輸入程式.xlsx")

In [3]:
import pandas as pd

nnl=pd.read_excel("/content/drive/MyDrive/大學/雲端-野村投信/野村持股(修改版)/NN (L)_修改版.xlsx")

In [6]:
##將欄位名稱更改為一致
dcard_WS.rename(columns={'time': 'publishAt','content': 'CONTENT_CKIP'}, inplace=True)
ptt_WS.rename(columns={'time': 'publishAt','content': 'CONTENT_CKIP'}, inplace=True)

###取出持股關鍵字之新聞

In [9]:
## 取出提到持股關鍵字新聞資料
## company 為持股資料
def headline_organize(company):
  pd.options.mode.chained_assignment = None  # default='warn'
  company_headline=pd.DataFrame()
  company_dcard=pd.DataFrame()
  company_ptt=pd.DataFrame()
  company_stock=pd.DataFrame()

  for j in range(len(company)):
    for i in company.loc[j].unique():
      if type(i)!=float:
        #headline
        data_headline=headline[headline['CONTENT'].str.lower().str.contains(i.lower())]
        data_headline['fund']=company.loc[j]['fund']
        company_headline=company_headline.append(data_headline)
        #dcard
        data_dcard=dcard_WS[dcard_WS['CONTENT_CKIP'].str.lower().str.contains(i.lower())]
        data_dcard['fund']=company.loc[j]['fund']
        company_dcard=company_dcard.append(data_dcard)
        #ptt
        data_ptt=ptt_WS[ptt_WS['CONTENT_CKIP'].str.lower().str.contains(i.lower())]
        data_ptt['fund']=company.loc[j]['fund']
        company_ptt=company_ptt.append(data_ptt)
        #stock
        data_stock=headline[headline['CONTENT'].str.lower().str.contains(i.lower())]
        data_stock['fund']=company.loc[j]['fund']
        company_stock=company_stock.append(data_stock)
  
  ##合併所有新聞及論壇至同個檔案
  df1 = pd.DataFrame({'publishAt': company_headline['publishAt'],
            'CONTENT_CKIP': company_headline['CONTENT_CKIP'],
            'fund': company_headline['fund']})
  df2 = pd.DataFrame({'publishAt': company_dcard['publishAt'],
              'CONTENT_CKIP': company_dcard['CONTENT_CKIP'],
              'fund': company_dcard['fund']})
  df3 = pd.DataFrame({'publishAt': company_ptt['publishAt'],
              'CONTENT_CKIP': company_ptt['CONTENT_CKIP'],
              'fund': company_ptt['fund']})
  df4 = pd.DataFrame({'publishAt': company_stock['publishAt'],
              'CONTENT_CKIP': company_stock['CONTENT_CKIP'],
              'fund': company_stock['fund']})
  frames = [df1, df2, df3, df4]
  company_all_news = pd.concat(frames)
  company_all_news.drop_duplicates(inplace = True) ## 重複的去除
  return company_all_news

In [8]:
nnl_news=headline_organize(nnl)
# abran_news=headline_organize(abran)
# ninetyone_news=headline_organize(ninetyone)
# ubsag_news=headline_organize(ubsag)
# roboco_news=headline_organize(roboco)
# ireland_news=headline_organize(ireland)
# nomura_news=headline_organize(nomura)
# amundi_news=headline_organize(amundi)
# janus_news=headline_organize(janus)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


###情緒分析

In [13]:
## 情緒字典
words=moods.values.tolist()
positive_words = []
negative_words = []
for i in words:
  if i[0]=='正向(中)' or i[0]=='正向(英)':
    positive_words.append(i[1])
  else:
    negative_words.append(i[1])

In [14]:
## 情緒分析
## company_news 為上方得到的持股相關新聞資料
def News_Sentiment(company_news):
  c=0
  try:
    company_news.insert(3,'positive',0, allow_duplicates=False)
    company_news.insert(4,'negative',0, allow_duplicates=False)
  except:
    pass
  for i in company_news['CONTENT_CKIP']:
    words=eval(i)
    positive = 0
    negative = 0
    for word in words:
      if word in positive_words:
        positive += 1
      elif word in negative_words:
        negative += 1
      else:
        pass
    
    ## 將正負向加入至表格中
    company_news['positive'][c]=positive
    company_news['negative'][c]=negative
    if c<len(company_news):
      c+=1
  company_news.reset_index(inplace=True)

In [15]:
News_Sentiment(nnl_news)
News_Sentiment(abran_news)
News_Sentiment(ninetyone_news)
News_Sentiment(ubsag_news)
News_Sentiment(roboco_news)
News_Sentiment(ireland_news)
News_Sentiment(nomura_news)
News_Sentiment(amundi_news)
News_Sentiment(janus_news)

In [ ]:
nnl_news.to_csv('/content/drive/MyDrive/大學/雲端-野村投信/情緒分析/NN(L)_情緒版.csv',encoding='utf-8', index=False)
abran_news.to_csv('/content/drive/MyDrive/大學/雲端-野村投信/情緒分析/安本標準_情緒版.csv',encoding='utf-8', index=False)
ninetyone_news.to_csv('/content/drive/MyDrive/大學/雲端-野村投信/情緒分析/晉達_情緒版.csv',encoding='utf-8', index=False)
ubsag_news.to_csv('/content/drive/MyDrive/大學/雲端-野村投信/情緒分析/瑞銀_情緒版.csv',encoding='utf-8', index=False)
roboco_news.to_csv('/content/drive/MyDrive/大學/雲端-野村投信/情緒分析/荷寶_情緒版.csv',encoding='utf-8', index=False)
ireland_news.to_csv('/content/drive/MyDrive/大學/雲端-野村投信/情緒分析/野村基金(愛爾蘭系列)_情緒版.csv',encoding='utf-8', index=False)
nomura_news.to_csv('/content/drive/MyDrive/大學/雲端-野村投信/情緒分析/野村投信_情緒版.csv',encoding='utf-8', index=False)
amundi_news.to_csv('/content/drive/MyDrive/大學/雲端-野村投信/情緒分析/鋒裕匯理_情緒版.csv',encoding='utf-8', index=False)
janus_news.to_csv('/content/drive/MyDrive/大學/雲端-野村投信/情緒分析/駿利亨德森_情緒版.csv',encoding='utf-8', index=False)

### 以日期跟基金名稱算總情緒分數

In [17]:
def result(company_news):
  import numpy as np
  from datetime import datetime

  dates = pd.to_datetime(company_news['publishAt'], format='%Y%m%d %H:%M:%S')
  company_news['publishAt']=dates.dt.strftime('%Y-%m-%d')

  company_final=pd.DataFrame({'publishAt':company_news['publishAt'],
              'fund':company_news['fund'],
              'positive':company_news['positive'],
              'negative':company_news['negative']})
  company_data=company_final.groupby(['fund','publishAt']).sum()
  return company_data

In [18]:
nnl_final=result(nnl_news)
abran_final=result(abran_news)
ninetyone_final=result(ninetyone_news)
ubsag_final=result(ubsag_news)
roboco_final=result(roboco_news)
ireland_final=result(ireland_news)
nomura_final=result(nomura_news)
amundi_final=result(amundi_news)
janus_final=result(janus_news)

In [19]:
nnl_final

positive  \
fund                                              publishAt              
NN (L) 亞洲債券基金\n(本基金有相當比重投資於非投資等級之高風險債券且配息來源可能為本金) 2021-10-25         0   
                                                  2021-11-02         0   
                                                  2022-02-25         0   
NN (L) 亞洲收益基金\n(本基金之配息來源可能為本金)                    2021-08-19        23   
                                                  2021-08-20       116   
...                                                                ...   
NN (L) 食品飲料基金\n(本基金之配息來源可能為本金)                    2022-01-26         0   
                                                  2022-02-09         0   
                                                  2022-02-10         0   
                                                  2022-02-17         0   
                                                  2022-02-23         0   

                                                              negative  
fund                                              publishAt             
NN (L) 亞洲債券基金\n(本基金有相當比重投資於非投資等級之高風險債券且配息來源可能為本金) 2021-10-25         0  
                                                  2021-11-02         0  
                                                  2022-02-25         0  
NN (L) 亞洲收益基金\n(本基金之配息來源可能為本金)                    2021-08-19         7  
                                                  2021-08-20        30  
...                                                                ...  
NN (L) 食品飲料基金\n(本基金之配息來源可能為本金)                    2022-01-26         0  
                                                  2022-02-09         0  
                                                  2022-02-10         0  
                                                  2022-02-17         0  
                                                  2022-02-23         0  

[1308 rows x 2 columns]

In [ ]:
nnl_news.groupby(['fund','publishAt']).sum().reset_index()[['fund', 'publishAt','positive','negative']].to_csv('/content/drive/MyDrive/大學/雲端-野村投信/整理完畢/NN(L)_按日期版.csv',encoding='utf-8', index=False)
abran_news.groupby(['fund','publishAt']).sum().reset_index()[['fund', 'publishAt','positive','negative']].to_csv('/content/drive/MyDrive/大學/雲端-野村投信/整理完畢/安本標準_按日期版.csv',encoding='utf-8', index=False)
ninetyone_news.groupby(['fund','publishAt']).sum().reset_index()[['fund', 'publishAt','positive','negative']].to_csv('/content/drive/MyDrive/大學/雲端-野村投信/整理完畢/晉達_按日期版.csv',encoding='utf-8', index=False)
ubsag_news.groupby(['fund','publishAt']).sum().reset_index()[['fund', 'publishAt','positive','negative']].to_csv('/content/drive/MyDrive/大學/雲端-野村投信/整理完畢/瑞銀_按日期版.csv',encoding='utf-8', index=False)
roboco_news.groupby(['fund','publishAt']).sum().reset_index()[['fund', 'publishAt','positive','negative']].to_csv('/content/drive/MyDrive/大學/雲端-野村投信/整理完畢/荷寶_按日期版.csv',encoding='utf-8', index=False)
ireland_news.groupby(['fund','publishAt']).sum().reset_index()[['fund', 'publishAt','positive','negative']].to_csv('/content/drive/MyDrive/大學/雲端-野村投信/整理完畢/野村基金(愛爾蘭系列)_按日期版.csv',encoding='utf-8', index=False)
nomura_news.groupby(['fund','publishAt']).sum().reset_index()[['fund', 'publishAt','positive','negative']].to_csv('/content/drive/MyDrive/大學/雲端-野村投信/整理完畢/野村投信_按日期版.csv',encoding='utf-8', index=False)
amundi_news.groupby(['fund','publishAt']).sum().reset_index()[['fund', 'publishAt','positive','negative']].to_csv('/content/drive/MyDrive/大學/雲端-野村投信/整理完畢/鋒裕匯理_按日期版.csv',encoding='utf-8', index=False)
janus_news.groupby(['fund','publishAt']).sum().reset_index()[['fund', 'publishAt','positive','negative']].to_csv('/content/drive/MyDrive/大學/雲端-野村投信/整理完畢/駿利亨德森_按日期版.csv',encoding='utf-8', index=False)